In [16]:
from dotenv import load_dotenv
import os
import pinecone
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")


In [8]:
pc= Pinecone(api_key=PINECONE_API_KEY)
pc.create_index(name="rate-my-professor", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1"))

In [13]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emma Thompson',
  'subject': 'Physics',
  'stars': 5,
  'review': "Dr. Thompson's lectures are engaging and she explains complex concepts clearly. Always available for office hours."},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 4,
  'review': 'Challenging coursework, but Prof. Chen is helpful when you ask questions. Great real-world examples.'},
 {'professor': 'Dr. Sarah Johnson',
  'subject': 'Psychology',
  'stars': 3,
  'review': 'Lectures can be dry, but the content is interesting. Wish there were more interactive elements in class.'},
 {'professor': 'Prof. David Lee',
  'subject': 'Mathematics',
  'stars': 5,
  'review': "Prof. Lee's passion for math is contagious. His problem-solving techniques have been invaluable."},
 {'professor': 'Dr. Rachel Green',
  'subject': 'Biology',
  'stars': 4,
  'review': "Engaging labs and field trips. Dr. Green's research experience adds depth to the course material."},
 {'professor': 'Prof

In [27]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(input=review['review'], model="text-embedding-3-small")
    embedding = response.data[0].embedding
    processed_data.append({
         "values": embedding,
         "id": review['professor'],
         "metadata": {
            "review":review['review'],
            "subject": review['subject'],
            "stars": review['stars'],
         }

         })

# pc.upsert(index="rate-my-professor", vectors=processed_data)

In [28]:
index = pc.Index("rate-my-professor")

index.upsert(
    vectors=processed_data,
    namespace="ns1"
)


{'upserted_count': 20}

In [29]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}